In [217]:
from importlib.metadata import version


pkgs = [
    "matplotlib","torch","numpy","tiktoken", "tensorflow"
]

[print(f"{pkg } \t: {version(pkg)}") for pkg in pkgs ];

matplotlib 	: 3.10.0
torch 	: 2.8.0+cu126
numpy 	: 2.0.2
tiktoken 	: 0.12.0
tensorflow 	: 2.19.0


In [218]:
import sys
import torch
sys.path.append('sample_data/')
from model_components_all import GPTModel


In [219]:

GPT_CONFIG_124M = {
    "vocab_size"  : 50257,
    "context_len" : 4,#256,
    "embd_dim"    : 768,
    "num_heads"   : 12,
    "num_layers"  : 12,
    "dropout_rate": 0.,
    "qkv_bias"    : False,
}


In [220]:
import tiktoken

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval();
tokenizer = tiktoken.get_encoding("gpt2")

In [221]:
# Text to tokenids
def text_to_tokenids(text, tokenizer):
  tok_ids = tokenizer.encode(text)
  token_ids = torch.tensor(tok_ids).unsqueeze(0) # add batch dim
  return token_ids

# Tokenids to text
def tokenids_to_text(tokenids, tokenizer):
  text = tokenizer.decode(tokenids.squeeze(0).tolist()) #remove batch dim
  return text

In [222]:
text1 = "Every effort moves you"
tok_ids = text_to_tokenids(text1, tokenizer)
tok_ids

tensor([[6109, 3626, 6100,  345]])

In [223]:
tokenids_to_text(tok_ids, tokenizer)

'Every effort moves you'

In [224]:
def generate_text(model, idx, num_samples, ctx_len):
  for _ in range(num_samples):
      idx_curr = idx[:,-ctx_len:]
      with torch.no_grad():
          logits= model(idx_curr)
      #print(f"{logits.shape}")
      idx_pred = logits[:,-1,:]
      #Extract the position index of the largest logits
      pred_tok = torch.argmax(idx_pred, dim=-1, keepdim=True)

      #print(tokenizer.decode(idx.squeeze(0).tolist()))
      idx= torch.cat((idx,pred_tok),dim=1)

  return idx

In [229]:
output = generate_text(model,tok_ids,10,4)
print(tokenids_to_text(output,tokenizer))

Every effort moves you naive Peerkitesley Nottingham formulation�TOP IT saint
